In [1]:
#| default_exp run
#|  export
from fastcore.script import call_parse
from torchtext.data.utils import get_tokenizer

def yield_tokens(
        data_iter # Iterable database instance
):
    tokenizer = get_tokenizer('basic_english')
    for _, text in data_iter:
        yield tokenizer(text)

def split_string(string):
    parts = string[1:-1].split(",")
    parts = [part.strip().strip("'") for part in parts]
    return parts[0], parts[1]

def return_iters(db:str # Path to db
                 ):
    train_iter = []
    test_iter = []
    file = open(db, 'r', encoding='latin1')
    mapping = {
        "Libertarian Left": 1,
        "Libertarian Right": 2,
        "Authoritarian Left": 3,
        "Authoritarian Right": 4,
        "Centrist": 5,
        "Authoritarian Center": 6,
        "Left": 7,
        "Right": 8,
        "Libertarian Center": 9,
    }
    lines = file.readlines()
    for line in lines:
        opinion,text = split_string(line)
        train_iter+=[(mapping[opinion],text)]
        test_iter+=[(mapping[opinion],text)]
    train_iter = iter(train_iter)
    test_iter = iter(test_iter)
    file.close()
    return train_iter, test_iter

In [2]:
#|  export
from torchtext.data.utils import get_tokenizer
from Political_Compass_AI.model import TextClassificationModel
from Political_Compass_AI.training import train
from Political_Compass_AI.training import evaluate
from torchtext.data.functional import to_map_style_dataset
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import time
import torch
import optuna
from optuna.trial import TrialState
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import pandas as pd
def define_model(trial,vocab_size, emsize, num_class):
    model = TextClassificationModel(vocab_size, emsize, num_class)
    return model
def collate_batch(
        batch
):
    global text_pipeline
    global db
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    label_pipeline = lambda x: int(x) - 1
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

def objective(
    trial,

):
    global text_pipeline
    global db
    BATCH_SIZE = trial.suggest_int('n_batch_size',32,128,32)
    db="../194511_DB_Hot_Top_New"
    tokenizer = get_tokenizer('basic_english')
    text_pipeline = lambda x: vocab(tokenizer(x))
    label_pipeline = lambda x: int(x) - 1
    train_iter, test_iter = return_iters(db)
    vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
    vocab.set_default_index(vocab["<unk>"])
    train_iter, test_iter = return_iters(db)
    dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)
    train_iter, test_iter = return_iters(db)
    num_class = len(set([label for (label, text) in train_iter]))
    vocab_size = len(vocab)
    emsize = trial.suggest_int("em_size",64,128,32)
    LR = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = define_model(trial,vocab_size, emsize, num_class).to(device)
    run_ledger = open("Run_Ledger.txt", 'a')
    criterion = torch.nn.CrossEntropyLoss()
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD","Adagrad"])
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
    _optim = optimizer_name
    total_accu = None
    train_iter, test_iter = return_iters(db)
    train_dataset = to_map_style_dataset(train_iter)
    test_dataset = to_map_style_dataset(test_iter)
    num_train = int(len(train_dataset) * 0.95)
    split_train_, split_valid_ = \
        random_split(train_dataset, [num_train, len(train_dataset) - num_train])

    train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                                  shuffle=True, collate_fn=collate_batch)
    valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                                  shuffle=True, collate_fn=collate_batch)
    test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                                 shuffle=True, collate_fn=collate_batch)
    EPOCHS = 20
    for epoch in range(1, EPOCHS + 1):
        epoch_start_time = time.time()
        train(train_dataloader, model, optimizer, epoch)
        accu_val = evaluate(valid_dataloader, model)
        if total_accu is not None and total_accu > accu_val:
            scheduler.step()
        else:
            total_accu = accu_val
        print('-' * 59)
        print('| end of epoch {:3d} | time: {:5.2f}s | '
              'valid accuracy {:8.3f} '.format(epoch,
                                               time.time() - epoch_start_time,
                                               accu_val))
        print('-' * 59)
        trial.report(accu_val, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    df_Log = {"Database_file":[],"Epochs":[],"LR":[],"Batch_Size":[],
              "Final_accu":[],"Optimzer":[],"accu_test":[]}
    accu_test = evaluate(test_dataloader,model)
    out = 'test accuracy {:8.3f}'.format(accu_test)
    df_Log["Database_file"].append(db)
    df_Log["Epochs"].append(str(EPOCHS))
    df_Log["LR"].append( str(LR))
    df_Log["Batch_Size"].append(str(BATCH_SIZE))
    df_Log["Final_accu"].append(str(accu_val))
    df_Log["Optimzer"].append(optimizer_name)
    df_Log["accu_test"].append(accu_test)
    dataframe = pd.DataFrame(df_Log)
    dataframe.to_csv('Run_Ledger.csv',mode='a', index=False,sep="\t")
    return accu_val

In [3]:
from optuna.pruners import MedianPruner
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=None)
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-04-30 06:58:10,381] A new study created in memory with name: no-name-61154ac3-e434-4679-98d2-0ada58288151


| epoch   1 |    50/ 5775 batches | accuracy    0.123
| epoch   1 |   100/ 5775 batches | accuracy    0.159
| epoch   1 |   150/ 5775 batches | accuracy    0.149
| epoch   1 |   200/ 5775 batches | accuracy    0.169
| epoch   1 |   250/ 5775 batches | accuracy    0.159
| epoch   1 |   300/ 5775 batches | accuracy    0.162
| epoch   1 |   350/ 5775 batches | accuracy    0.150
| epoch   1 |   400/ 5775 batches | accuracy    0.153
| epoch   1 |   450/ 5775 batches | accuracy    0.158
| epoch   1 |   500/ 5775 batches | accuracy    0.157
| epoch   1 |   550/ 5775 batches | accuracy    0.163
| epoch   1 |   600/ 5775 batches | accuracy    0.159
| epoch   1 |   650/ 5775 batches | accuracy    0.180
| epoch   1 |   700/ 5775 batches | accuracy    0.168
| epoch   1 |   750/ 5775 batches | accuracy    0.150
| epoch   1 |   800/ 5775 batches | accuracy    0.159
| epoch   1 |   850/ 5775 batches | accuracy    0.173
| epoch   1 |   900/ 5775 batches | accuracy    0.182
| epoch   1 |   950/ 5775 ba

[I 2023-04-30 07:04:54,826] Trial 0 finished with value: 0.18465967509767633 and parameters: {'n_batch_size': 32, 'em_size': 64, 'lr': 0.0005042608061319348, 'optimizer': 'RMSprop'}. Best is trial 0 with value: 0.18465967509767633.


| epoch   1 |    50/ 2888 batches | accuracy    0.111
| epoch   1 |   100/ 2888 batches | accuracy    0.116
| epoch   1 |   150/ 2888 batches | accuracy    0.107
| epoch   1 |   200/ 2888 batches | accuracy    0.116
| epoch   1 |   250/ 2888 batches | accuracy    0.112
| epoch   1 |   300/ 2888 batches | accuracy    0.113
| epoch   1 |   350/ 2888 batches | accuracy    0.109
| epoch   1 |   400/ 2888 batches | accuracy    0.113
| epoch   1 |   450/ 2888 batches | accuracy    0.102
| epoch   1 |   500/ 2888 batches | accuracy    0.107
| epoch   1 |   550/ 2888 batches | accuracy    0.099
| epoch   1 |   600/ 2888 batches | accuracy    0.111
| epoch   1 |   650/ 2888 batches | accuracy    0.110
| epoch   1 |   700/ 2888 batches | accuracy    0.116
| epoch   1 |   750/ 2888 batches | accuracy    0.110
| epoch   1 |   800/ 2888 batches | accuracy    0.115
| epoch   1 |   850/ 2888 batches | accuracy    0.120
| epoch   1 |   900/ 2888 batches | accuracy    0.105
| epoch   1 |   950/ 2888 ba

[I 2023-04-30 07:08:42,689] Trial 1 finished with value: 0.11207073822743163 and parameters: {'n_batch_size': 64, 'em_size': 128, 'lr': 2.9234770280833206e-05, 'optimizer': 'SGD'}. Best is trial 0 with value: 0.18465967509767633.


...

-----------------------------------------------------------
| end of epoch   1 | time: 10.43s | valid accuracy    0.180 
-----------------------------------------------------------
| epoch   1 |    50/ 5775 batches | accuracy    0.133
| epoch   1 |   100/ 5775 batches | accuracy    0.146
| epoch   1 |   150/ 5775 batches | accuracy    0.161
| epoch   1 |   200/ 5775 batches | accuracy    0.159
| epoch   1 |   250/ 5775 batches | accuracy    0.166
| epoch   1 |   300/ 5775 batches | accuracy    0.165
| epoch   1 |   350/ 5775 batches | accuracy    0.149
| epoch   1 |   400/ 5775 batches | accuracy    0.177
| epoch   1 |   450/ 5775 batches | accuracy    0.166
| epoch   1 |   500/ 5775 batches | accuracy    0.168
| epoch   1 |   550/ 5775 batches | accuracy    0.154
| epoch   1 |   600/ 5775 batches | accuracy    0.173
| epoch   1 |   650/ 5775 batches | accuracy    0.183
| epoch   1 |   700/ 5775 batches | accuracy    0.162
| epoch   1 |   750/ 5775 batches | accuracy    0.159
| epoch  

[I 2023-04-30 10:00:53,716] Trial 98 pruned. 


-----------------------------------------------------------
| end of epoch   1 | time: 19.46s | valid accuracy    0.179 
-----------------------------------------------------------
| epoch   1 |    50/ 5775 batches | accuracy    0.149
| epoch   1 |   100/ 5775 batches | accuracy    0.169
| epoch   1 |   150/ 5775 batches | accuracy    0.154
| epoch   1 |   200/ 5775 batches | accuracy    0.170
| epoch   1 |   250/ 5775 batches | accuracy    0.164
| epoch   1 |   300/ 5775 batches | accuracy    0.156
| epoch   1 |   350/ 5775 batches | accuracy    0.177
| epoch   1 |   400/ 5775 batches | accuracy    0.154
| epoch   1 |   450/ 5775 batches | accuracy    0.152
| epoch   1 |   500/ 5775 batches | accuracy    0.176
| epoch   1 |   550/ 5775 batches | accuracy    0.171
| epoch   1 |   600/ 5775 batches | accuracy    0.175
| epoch   1 |   650/ 5775 batches | accuracy    0.149
| epoch   1 |   700/ 5775 batches | accuracy    0.168
| epoch   1 |   750/ 5775 batches | accuracy    0.158
| epoch  

[I 2023-04-30 10:01:16,476] Trial 99 pruned. 


-----------------------------------------------------------
| end of epoch   1 | time: 19.68s | valid accuracy    0.179 
-----------------------------------------------------------
Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  80
  Number of complete trials:  20
Best trial:
  Value:  0.1958667489204195
  Params: 
    n_batch_size: 32
    em_size: 64
    lr: 0.0004774035163878788
    optimizer: RMSprop
